In [51]:
import json
import pandas as pd

In [52]:
# Load the JSON log file
file_path = '<path to log file>'

In [ ]:
with open(file_path, 'r') as file:
    log_data = json.load(file)

# Display the first few entries to understand the structure
log_data[:2]

In [54]:
# Define the fields to extract
fields_to_extract = ['tag', 'eventType', 'eventId', 'category', 'service', 'timestamp']
context_fields_to_extract = ['measurement', 'unitType']
correlation_fields_to_extract = ['requestT0', 'sessionId', 'actionId', 'customId']

In [55]:
# Extract the specified fields from each log event
extracted_data = []
for event in log_data:
    extracted_event = {field: event.get(field, None) for field in fields_to_extract}
    
    # Check for the context node inside payload and extract its fields
    if 'payload' in event and 'context' in event['payload']:
        context = event['payload']['context']
        for field in context_fields_to_extract:
            extracted_event[field] = context.get(field, None)
    else:
        # If context is not present, set the fields to None
        for field in context_fields_to_extract:
            extracted_event[field] = None
            
    # Check for the correlationId node and extract its fields
    if 'correlationId' in event:
        try:
            correlation_id = json.loads(event['correlationId'])
            for field in correlation_fields_to_extract:
                value = correlation_id.get(field, None)
                if field == 'requestT0' and value is not None:
                    # Convert requestT0 to string to prevent exponential form
                    extracted_event[field] = str(value)
                else:
                    extracted_event[field] = value
        except json.JSONDecodeError:
            for field in correlation_fields_to_extract:
                extracted_event[field] = None
    else:
        # If correlationId is not present, set the fields to None
        for field in correlation_fields_to_extract:
            extracted_event[field] = None
            
    extracted_data.append(extracted_event)

In [56]:
# Convert the extracted data into a pandas DataFrame
df = pd.DataFrame(extracted_data)

In [57]:
# Save the DataFrame to an Excel file
output_file_path = '<path>/log-parser/extracted_log_data.xlsx'
df.to_excel(output_file_path, index=False)

In [ ]:
print(f"Extracted data has been saved to {output_file_path}")